In [76]:
import numpy as np
import pandas as pd
import os
import json
import spacy
from itertools import chain, count
from collections import Counter
import re

In [5]:
RC_ROOT = "/home/mithrillion/Data_Drive/Projects/text_reading/data/processed/cnn/"
es_pkl = "/home/mithrillion/Data_Drive/Projects/BiQA/cnn_es.pkl"

In [6]:
es_dat = pd.read_pickle(es_pkl)

In [7]:
es_dat.head()

,headline,keyfacts,content,tags,time
http://cnnespanol.cnn.com/2015/05/05/estudiantes-peruanos-ensenan-historia-en-los-autobuses/,Estudiantes peruanos enseñan historia... en lo...,[Un grupo de estudiantes de la Universidad San...,(CNN Español) - En medio de los tediosos y caó...,[],"(02:22 GMT) 5 mayo, 2015"
http://cnnespanol.cnn.com/2015/05/05/la-dea-sanciona-a-agentes-que-olvidaron-a-estudiante-en-una-celda-por-cinco-dias/,La DEA sanciona a agentes que olvidaron a estu...,[Daniel Chong fue esposado y mantenido dentro ...,(CNN) – La Administración para el Control de D...,"[Daniel Chong, DEA]","(03:09 GMT) 5 mayo, 2015"
http://cnnespanol.cnn.com/2015/05/06/el-papa-francisco-aprende-trucos-de-basquetbol-con-los-harlem-globetrotters/,El papa Francisco aprende trucos de basquetbol...,[Francisco es la novena persona en la historia...,(CNNMéxico) — ¿Un papa basquetbolista? No exac...,"[Harlem Globetrotters, Papa Francisco]","(01:23 GMT) 6 mayo, 2015"
http://cnnespanol.cnn.com/2015/05/06/es-momento-de-un-cambio-michelle-bachelet-le-pide-la-renuncia-a-su-gabinete/,'Es momento de un cambio': Michelle Bachelet l...,[Michelle Bachelet anunció que pidió la renunc...,(CNN Español) - La presidenta Michelle Bachele...,[Michele Bachelet],"(01:33 GMT) 6 mayo, 2015"
http://cnnespanol.cnn.com/2015/05/06/partidos-politicos-de-mexico-adoptan-medidas-para-lidiar-con-la-violencia-contra-candidatos/,Partidos políticos de México adoptan medidas p...,[Una serie de hechos de violencia registrados ...,(CNN Español) - El Partido Acción Nacional de ...,[],"(01:29 GMT) 6 mayo, 2015"


In [8]:
(es_dat['keyfacts'].apply(len) == 0).sum()

0

In [19]:
print(es_dat['keyfacts'][2])

['Francisco es la novena persona en la historia en ser convertido en Globetrotter honorario', 'El legendario equipo neoyorquino celebra sus 90 años con una gira internacional']


In [10]:
nlp = spacy.load('es')

In [11]:
nlp.pipeline

In [26]:
es_entlist = es_dat['keyfacts'].apply(lambda l: [nlp(s).ents for s in l])

In [57]:
es_ents = [list(chain.from_iterable([[w.text for w in d] for d in l])) for l in es_entlist]

In [58]:
es_ents[:5]

[['Universidad San Martín de'],
 ['Daniel Chong', '4.1 millones', 'DEA'],
 ['Francisco', 'Globetrotter', '90'],
 ['Michelle Bachelet', '72', 'Bachelet'],
 ['Guerrero', '7', '2.000', '9', 'Guerrero', 'Cámara de Diputados']]

In [60]:
cnt = Counter(chain.from_iterable(es_ents))

In [61]:
cnt.most_common(10)

[('Trump', 284),
 ('Estados Unidos', 188),
 ('dos', 181),
 ('uno', 104),
 ('primera', 90),
 ('tres', 85),
 ('Donald Trump', 80),
 ('México', 79),
 ('EE.UU', 75),
 ('Venezuela', 70)]

In [62]:
len(cnt)

4856

In [64]:
np.sum([len(x) for x in es_ents])

11248

In [69]:
def is_number(x):
    if re.search(r'\d+', x):
        return True
    else:
        return False

In [87]:
nums = pd.DataFrame([list(cnt.keys()), list(map(is_number, list(cnt.keys())))]).T
nums.columns = ['entity', 'is_number']

In [88]:
nums.head(5)

,entity,is_number
0,Universidad San Martín de,False
1,Daniel Chong,False
2,4.1 millones,True
3,DEA,False
4,Francisco,False


In [89]:
num_id = count()
ent_id = count()
nums['replacement'] = np.nan
for i in range(len(nums)):
    if nums.iloc[i, 1]:
        nums.iloc[i, 2] = "@number{0}".format(next(num_id))
    else:
        nums.iloc[i, 2] = "@entity{0}".format(next(ent_id))

In [90]:
nums.head(10)

,entity,is_number,replacement
0,Universidad San Martín de,False,@entity0
1,Daniel Chong,False,@entity1
2,4.1 millones,True,@number0
3,DEA,False,@entity2
4,Francisco,False,@entity3
5,Globetrotter,False,@entity4
6,90,True,@number1
7,Michelle Bachelet,False,@entity5
8,72,True,@number2
9,Bachelet,False,@entity6


In [91]:
replacement_dict = {k: v for k, v in zip(nums['entity'], nums['replacement'])}

{'Universidad San Martín de': '@entity0',
 'Daniel Chong': '@entity1',
 '4.1 millones': '@number0',
 'DEA': '@entity2',
 'Francisco': '@entity3',
 'Globetrotter': '@entity4',
 '90': '@number1',
 'Michelle Bachelet': '@entity5',
 '72': '@number2',
 'Bachelet': '@entity6',
 'Guerrero': '@entity7',
 '7': '@number3',
 '2.000': '@number4',
 '9': '@number5',
 'Cámara de Diputados': '@entity8',
 'Ministro de Salud': '@entity9',
 'Henry Ventura': '@entity10',
 'Federación Farmacéutica de Venezuela': '@entity11',
 '60': '@number6',
 'Gran Caracas': '@entity12',
 'Gobierno': '@entity13',
 'Mhairi Black': '@entity14',
 'Reino Unido': '@entity15',
 '1667': '@number7',
 '\xa0Douglas Alexander': '@entity16',
 'Partido Laborista': '@entity17',
 'Nintendo': '@entity18',
 'Universal': '@entity19',
 'Pele': '@entity20',
 '74': '@number8',
 'dos': '@entity21',
 'Sao Paulo': '@entity22',
 'Mars': '@entity23',
 'M&M': '@entity24',
 'Snickers': '@entity25',
 'Twix': '@entity26',
 '\xa0': '@entity27',
 'Klui

In [98]:
def dict_replacement(s, d):
    pattern = re.compile(r'\b(' + '|'.join(d.keys()) + r')\b')
    return pattern.sub(lambda x: d[x.group()], s)

In [101]:
dict_replacement(es_dat['keyfacts'][2][0], replacement_dict)

'@entity3 es la novena persona en la historia en ser convertido en @entity4 honorario'